<a href="https://colab.research.google.com/github/veryHapppy/study_ai/blob/main/Kaggle/TrafficSignNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
 95% 581M/612M [00:02<00:00, 172MB/s]
100% 612M/612M [00:03<00:00, 209MB/s]


In [ ]:
import numpy as np
import pandas as pd
import cv2
from PIL import Image

data = []
labels = []
classes = 43

for i in range(classes):
  path = os.path.join('train', str(i))
  images = os.listdir(path)

  for a in images:
    try:
      image = cv2.imread(os.path.join(path, a))
      image_from_array = Image.fromarray(image, 'RGB')
      size_image = image_from_array.resize((30, 30))
      data.append(np.array(size_image))
      labels.append(i)
    except:
      print(f"Image Error :{a}")

X_train = np.array(data)
y_train = np.array(labels)

print(X_train.shape)

/tmp/ipython-input-3738056754.py:17: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  image_from_array = Image.fromarray(image, 'RGB')


(39209, 30, 30, 3)


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

X_train_pt = torch.FloatTensor(X_train).permute(0,3,1,2)
y_train_pt = torch.LongTensor(labels)

train_X, val_X, train_y, val_y = train_test_split(X_train_pt, y_train_pt, test_size=0.2, random_state=42)

train_dataset = TensorDataset(train_X, train_y)
val_dataset = TensorDataset(val_X, val_y)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class TrafficSignNet(nn.Module):
  def __init__(self):
    super(TrafficSignNet, self).__init__()

    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
    self.bn1 = nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.bn2 = nn.BatchNorm2d(64)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    self.fc1 = nn.Linear(64 * 15 * 15, 512)
    self.bn_fc = nn.BatchNorm1d(512)
    self.fc2 = nn.Linear(512, 43)

    self.dropout = nn.Dropout(0.25)

  def forward(self, x):
    x = F.relu(self.bn1(self.conv1(x)))
    x = self.pool(F.relu(self.bn2(self.conv2(x))))
    x = self.dropout(x)

    x = x.view(-1, 64 * 15 * 15)

    x = F.relu(self.bn_fc(self.fc1(x)))
    x = self.dropout(x)
    x = self.fc2(x)
    return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TrafficSignNet().to(device)

print(model)

TrafficSignNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=14400, out_features=512, bias=True)
  (bn_fc): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=43, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 20
history = {'train_loss': [], 'train_acc': [], 'val_loss':[], 'val_acc': []}

for epoch in range(epochs):
  model.train()
  train_loss = 0.0
  correct_train = 0
  total_train = 0

  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item()
    _, predicted = torch.max(outputs.data, 1)
    total_train += labels.size(0)
    correct_train += (predicted == labels).sum().item()

  model.eval()
  val_loss = 0.0
  correct_val = 0
  total_val = 0

  with torch.no_grad():
    for images, labels in val_loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels)

      val_loss += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      total_val += labels.size(0)
      correct_val += (predicted == labels).sum().item()

  train_acc = 100 * correct_train / total_train
  val_acc = 100 * correct_val / total_val

  history['train_loss'].append(train_loss/len(train_loader))
  history['train_acc'].append(train_acc)
  history['val_loss'].append(val_loss/len(val_loader))
  history['val_acc'].append(val_acc)

  print(f'Epoch [{epoch+1}/{epochs}] '
        f'Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%')

Epoch [1/20] Train Acc: 90.34% | Val Acc: 97.97%
Epoch [2/20] Train Acc: 98.11% | Val Acc: 98.41%
Epoch [3/20] Train Acc: 98.87% | Val Acc: 98.87%
Epoch [4/20] Train Acc: 99.03% | Val Acc: 99.11%
Epoch [5/20] Train Acc: 99.29% | Val Acc: 99.18%
Epoch [6/20] Train Acc: 99.29% | Val Acc: 99.09%
Epoch [7/20] Train Acc: 99.40% | Val Acc: 99.32%
Epoch [8/20] Train Acc: 99.50% | Val Acc: 99.35%
Epoch [9/20] Train Acc: 99.53% | Val Acc: 99.32%
Epoch [10/20] Train Acc: 99.48% | Val Acc: 99.27%
Epoch [11/20] Train Acc: 99.63% | Val Acc: 99.46%
Epoch [12/20] Train Acc: 99.73% | Val Acc: 99.49%
Epoch [13/20] Train Acc: 99.64% | Val Acc: 99.54%
Epoch [14/20] Train Acc: 99.52% | Val Acc: 99.43%
Epoch [15/20] Train Acc: 99.69% | Val Acc: 99.38%
Epoch [16/20] Train Acc: 99.78% | Val Acc: 99.43%
Epoch [17/20] Train Acc: 99.65% | Val Acc: 99.43%
Epoch [18/20] Train Acc: 99.79% | Val Acc: 99.36%
Epoch [19/20] Train Acc: 99.69% | Val Acc: 99.35%
Epoch [20/20] Train Acc: 99.74% | Val Acc: 99.63%


In [ ]:
test_df = pd.read_csv(('Test.csv'))
test_labels = test_df['ClassId'].values
test_img_paths = test_df['Path'].values

test_data = []

for img_path in test_img_paths:
  try:
    full_path = img_path
    image = cv2.imread(full_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_from_array = Image.fromarray(image)
    size_image = image_from_array.resize((30, 30))
    test_data.append(np.array(size_image))
  except:
    print(f"image Error: {img_path}")

X_test = np.array(test_data)
X_test_pt = torch.FloatTensor(X_test).permute(0,3,1,2)
y_test_pt = torch.LongTensor(test_labels)

test_dataset = TensorDataset(X_test_pt, y_test_pt)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
correct = 0
total = 0

with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'정확도: {100 * correct / total:.2f}%')

정확도: 79.87%


In [ ]:
print(len(test_data))

12630
